In [1]:
import math, random, sympy
from math import log
import string

---

#### **Extended Euclidean Algorithm (EEA)**

The `EED` function implements the Extended Euclidean Algorithm, which solves the equation:
$$
au + bv = d
$$
where $d = \gcd(a, b)$. It returns the tuple $(d, u, v)$, where:
- $d$ is the greatest common divisor of $a$ and $b$.
- $u$ and $v$ are the coefficients (Bézout coefficients) such that $au + bv = d$.


#### **Modular Inverse**

The `compute_inverse` function uses the EEA to compute the modular inverse of $a$
modulo $n$. If $a$ and $n$ are coprime (i.e., $\gcd(a, n) = 1$), it returns the
modular inverse $v \mod n$. Otherwise, it prints a message indicating that $a$ is not invertible.


---

In [2]:
def EED(a: int, b: int):
    """
    solve:
        au + bv = d
        d:= gcd(a, b)

    returns: (d, u, v)
    """
    u0, u1 = 1, 0
    v0, v1 = 0, 1
    r = 7
    while r != 0:
        q, r = a//b, a%b#divmod(a, b)
        u1, u0 = u0 - q*u1, u1
        v1, v0 = v0 - q*v1, v1
        a, b = b, r

    return a, u0, v0


def compute_inverse(a: int, n: int):
    """
    returns: - x where ax = 1 mod n
             - None if a and n is not coprime
    """
    d, u, v = EED(n, a)
    if d == 1:
        return v % n
    print(f"{a} is not invertible over Z/{n}Z")

def chinese_reminder(a1, a2, n1, n2):
    d, u, v = EED(n1, n2)
    if d == 1:
        N1 = n2
        m1 = v

        N2 = n1
        m2 = u
        return (a1*N1*m1 + a2*N2*m2) % (n1*n2)

    print("Solution doesn't exist.")


def pseudo_primality_test(N: int, x: int):

    """
    returns: - False (composite number)
             - True  (probably prime to base x)
    """
    e = 0
    N1 = N-1
    while N1 % 2 == 0:
        e += 1
        N1 = N1 // 2
    m = N1

    # x = random.randint(2, N-1)
    d, _, _ = EED(N, x)

    if d != 1:
        return False

    y = pow(x, m, N)
    if y == 1:
        return True

    for i in range(e):
        if y == N-1:
            return True
        else:
            y = pow(y, 2, N)

    return False

def primality_test(N: int, k: int=20):
    if N == 2:
        return True

    for _ in range(k):
        # x: witness value
        x = random.randint(1, N-1)

        if not pseudo_primality_test(N, x):
            return False

    return True

def generate_prime(n:int, prime_algo):
    """
    Inputs:
        - n: number of bits
        - prime_algo: primality test algorithm

    Output: 
        - Prime number with number of bits n
    """
    
    if n <= 3:
        n = 6
        
    y = pow(2, (n-1)/2)
    lw = math.floor(y) + 1
    up = math.floor(y*math.sqrt(2)) - 1
    while True:
        n = random.randint(lw, up)
        if prime_algo(n):
            return n

---



#### **RSA Key Generation**

The RSA key generation process involves:
1. Generating two large prime numbers $p$ and $q$.
2. Computing $N = p \times q$ and $\varphi(N) = (p-1)(q-1)$.
3. Selecting a public key $e$ such that $\gcd(e, \varphi(N)) = 1$.
4. Computing the private key $d$ as the modular inverse of $e$ modulo $\varphi(N)$.



In [3]:
def key_generation_rsa(p, q):
    N = p*q
    L = (p-1)*(q-1)
    while True:
        e = random.randint(2, L-1)
        d, u, v = EED(L, e)
        if d == 1:
            return N, e, v % L

#### **Encryption**
The `encryption` function encodes a message into a numerical format and encrypts it using the public key $(N, e)$.



#### **Decryption**
The `decryption` function decrypts the ciphertext using the private key $(N, d)$ and decodes it back into the original message.

In [4]:
BASE = string.printable


def convert_str_to_int(msg:str, b:int):
    x = 0
    for i in range(len(msg)):
        x += pow(b, i)*BASE.index(msg[i])
    return x

def convert_int_to_str(x:int, b:int):

    msg = ''
    while x>=b:
        msg += BASE[x%b]
        x //=b
    msg += BASE[x]
    return msg

def convert_base_n(number:int, b):
    l = []
    while number>=b:
        l.append(number%b)
        number //= b
    l.append(number)
    return l

def encryption(message:str, e: int, n: int):

    msg = convert_str_to_int(message, 100)
    base_n = convert_base_n(msg, n)
    val = 0
    for i in range(len(base_n)):
        val += pow(base_n[i], e, n)*n**i

    return convert_int_to_str(val, 100)

def decryption(message:str, d: int, n: int):

    msg = convert_str_to_int(message, 100)
    base_n = convert_base_n(msg, n)
    val = 0
    for i in range(len(base_n)):
        val += pow(base_n[i], d, n)*n**i

    return convert_int_to_str(val, 100)

#### **Use the two following safe primes for p and q.**
* *The security of RSA relies on the difficulty of factoring the product $N=pq$ into its prime factors q and p.*
* *A safe prime is a prime number of the form $p=2q+1$, where $q$ is also a prime*
* *A safe prime makes certain attacks, such as Pollard's $p−1$ factorization algorithm, less effective.*
* *They ensure that $p−1$ has a large prime factor, which strengthens the security of the modulus $N$.*

    

In [5]:
p = 49635124589134858096121251644242489628344943459974812371817232056198344606696117926977793428185221500635617430184806373870258348740214426279109660899463935660251185317353656423283576439282267019129259164154281909683475137370970673432196999445178922501041062966015556349739959413639042929250732476767538436234280073916069226106947826257158376209221119040271449066873955696813401291084079495124976916708786955922019555451823754590720260965596245097151742891885915202997713570931595783576565372604279311985499142754730544113767278628220983747830518503417833003537488693938267019628231805325287835718263

In [6]:
q = 74965465127118954159631720210745557201980041881006055614171953422712826523957449446036248892868765348553669818416270469792026622984136848797168921869409941943513445690626344313957312339747695972233508010058081885393013933459918865750408501321175581300415187525700294949282119611809486210993632264377587174092672902479867882803255613184166920581885635556894571419543582173700112297345200610557191836329086205956187111683375202777313555935565931955325103654757609521869668100519313437307958075931559106905592709203326532659764327922625682399072642067989554280046771916069679669240510574823200126496223

#### **Finding public key and secret key using the two safe primes.**

In [7]:
n, e, d = key_generation_rsa(q, p)

In [8]:
message = """THE ART OF BEING QUIETLY SILENT, ESTIMATED READER, IS A TRULY NOBLE THING; EVEN NOBLER
IS IN MY VIEW THE ART OF BEING SILENT WHILE TALKING, AND THIS ART IS TAUGHT TO YOU BY MY
CRYPTOGRAPHIA, OR THE ART OF MAKING HIDDEN LETTERS AND WRITINGS, BROUGHT HERE TO THE
LIGHT OF DAY IN OUR MOTHER TONGUE."""

print(message)

THE ART OF BEING QUIETLY SILENT, ESTIMATED READER, IS A TRULY NOBLE THING; EVEN NOBLER
IS IN MY VIEW THE ART OF BEING SILENT WHILE TALKING, AND THIS ART IS TAUGHT TO YOU BY MY
CRYPTOGRAPHIA, OR THE ART OF MAKING HIDDEN LETTERS AND WRITINGS, BROUGHT HERE TO THE
LIGHT OF DAY IN OUR MOTHER TONGUE.


In [10]:
enc_mesg = encryption(message, e, n)

print('Encrypted Message: ')
enc_mesg

Encrypted Message: 


'_A?yRE\x0cO\ne\r!._%h\\QA m>oPj^61k\r,EYrZ(t0Nr&5nK%RVz{RuwYEr87Jzr\rYW$a_WSi7U\tYNn"&%XwOC&YiF*tgH._a\x0c1GiXe;\x0c;nQH142x*\x0b07>$[|yKMm5Q+<y\\Ch;{*%?ypUy`m/6|v(rMq@{+D(U*Z63\t-Bt3\nJ<xvq((TO.CQ"$H)\rXf@|[{.v\'!k;rW;2:9rPe6aqx"F\r\t$b&-t2.fmBd@m:eYc8)+h\n16HMDpbDM|Hk1\tIL^}_}Gn~bR>\x0bF\'c4xL6%=3|P\\"\x0b51itdR#F\n<I\'m:op@HN8<*t>5EAl1\x0czwxljIf#sIZ8kgf\x0cv]f^,[UvM,&iQB7Z1>Uc\x0b\'-cbz;7\'=b*p^Jj$\'X\'?SNNn&:r8VSrKCe.9?l}l[DcV[d[\rUkyADZ,$`y%{K<s9FZ#m+FGq]R@Bh(d3fgkgJw\nm?\\^$?CC3$d1MeZ\'[YX@8FvYoCtI\n`</TD7{V/y7B\ry`/%Q,b_P$X|{O\x0b\t!N2&RVhd>A!C}6law,6lRnMh4J%%(ArpxJX2d06ZApC!#iAoQ#`;)\x0b^)<_+?#+\rs?YYx!S@wf~)2\r(hDN|%SP6IkeRB-\\qZ#4@<[C{5lpe~c}M\tfz'

In [11]:
print('Decrypted Message: ')
print(decryption(enc_mesg, d, n))

Decrypted Message: 
THE ART OF BEING QUIETLY SILENT, ESTIMATED READER, IS A TRULY NOBLE THING; EVEN NOBLER
IS IN MY VIEW THE ART OF BEING SILENT WHILE TALKING, AND THIS ART IS TAUGHT TO YOU BY MY
CRYPTOGRAPHIA, OR THE ART OF MAKING HIDDEN LETTERS AND WRITINGS, BROUGHT HERE TO THE
LIGHT OF DAY IN OUR MOTHER TONGUE.
